[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RayZhhh/py-adtools/blob/main/tutorial/04_evaluator.ipynb)

In [ ]:
# Install the library if you are running this in Colab
!pip install py-adtools


# adtools.evaluator 教程：代码评估框架

`adtools.evaluator` 提供了一个结构化的框架，用于评估生成的 Python 程序。它结合了代码解析和沙盒执行，使我们能够安全地测试算法的正确性和性能。

## 核心概念

`PyEvaluator` 是一个抽象基类。要使用它，你需要继承它并实现 `evaluate_program` 方法。这个方法定义了如何评估一段特定的代码。

## 实现自定义评估器

下面我们将实现一个用于评估“排序算法”的评估器。


In [ ]:
import time
from typing import Dict, Callable, List, Any
from adtools.evaluator import PyEvaluator

class SortAlgorithmEvaluator(PyEvaluator):
    def evaluate_program(
            self,
            program_str: str,
            callable_functions_dict: Dict[str, Callable] | None,
            callable_functions_list: List[Callable] | None,
            callable_classes_dict: Dict[str, Callable] | None,
            callable_classes_list: List[Callable] | None,
            **kwargs,
    ) -> Any | None:
        """评估给定的排序算法程序。"""
        
        # 1. 获取目标函数
        # 假设生成的代码中应该包含一个名为 'merge_sort' 的函数
        sort_algo = callable_functions_dict.get("merge_sort")
        if not sort_algo:
            return None  # 如果没有找到目标函数，评估失败

        # 2. 准备测试数据
        input_data = [10, 2, 4, 76, 19, 29, 3, 5, 1]
        expected_output = sorted(input_data)

        # 3. 执行并计时
        start = time.time()
        try:
            res = sort_algo(input_data[:]) # 传入副本以防修改原数据
        except Exception:
            return None # 代码抛出异常
            
        duration = time.time() - start

        # 4. 验证结果
        if res == expected_output:
            # 如果结果正确，返回执行时间作为分数（越快越好）
            return duration
        else:
            return None  # 结果不正确


## 使用评估器

现在我们可以使用这个评估器来测试一些代码字符串。`PyEvaluator` 会自动处理代码的执行（exec）并将提取出的函数传递给你的 `evaluate_program` 方法。


In [ ]:
# 一个正确的归并排序实现
correct_code = """
def merge_sort(arr):
    if len(arr) <= 1: return arr
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    
    def merge(left, right):
        result = []
        i = j = 0
        while i < len(left) and j < len(right):
            if left[i] < right[j]:
                result.append(left[i])
                i += 1
            else:
                result.append(right[j])
                j += 1
        result.extend(left[i:])
        result.extend(right[j:])
        return result

    return merge(left, right)
"""

# 一个错误的实现
wrong_code = """
def merge_sort(arr):
    return arr # 没有排序
"""

# 初始化评估器
evaluator = SortAlgorithmEvaluator()

print("评估正确的代码...")
# _exec_and_get_res 是在当前进程直接执行，主要用于调试
score = evaluator._exec_and_get_res(correct_code)
print(f"得分 (耗时): {score}")

print("评估错误的代码...")
score = evaluator._exec_and_get_res(wrong_code)
print(f"得分: {score}")


## 安全评估

在实际应用中，我们通常使用 `secure_evaluate` 方法。这会在沙盒进程中运行评估逻辑，从而提供超时保护。


In [ ]:
infinite_loop_code = """
def merge_sort(arr):
    while True: pass
"""

print("安全评估无限循环代码...")
result = evaluator.secure_evaluate(infinite_loop_code, timeout_seconds=2)
print(f"结果: {result['result']}")
print(f"错误信息: {result['error_msg']}")


## PyEvaluatorRay

如果你的评估任务需要大量的计算资源（例如 GPU），或者返回的对象非常大（例如训练好的模型权重），建议继承 `PyEvaluatorRay`。

```python
from adtools.evaluator import PyEvaluatorRay

class DistributedEvaluator(PyEvaluatorRay):
    def evaluate_program(self, ...):
        # ...
        pass
```

用法与 `PyEvaluator` 完全一致，但底层的 `secure_evaluate` 会在 Ray Actor 中执行。
